In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Loading the sql module to ipython using line magic
%load_ext sql

In [3]:
%sql sqlite:///db.sqlite3

In [4]:
%sql customuser << SELECT * FROM users_customuser;
%sql ratings << SELECT * FROM main_app_ratings;
%sql books << SELECT * FROM main_app_books;

 * sqlite:///db.sqlite3
Done.
Returning data to local variable customuser
 * sqlite:///db.sqlite3
Done.
Returning data to local variable ratings
 * sqlite:///db.sqlite3
Done.
Returning data to local variable books


In [ ]:
customuserDf = pd.DataFrame(customuser)
ratingsDf = pd.DataFrame(ratings)
booksDf = pd.DataFrame(books)

In [ ]:
display(f"customuser: {customuserDf.columns}\n")
display(f"rating: {ratingsDf.columns}\n")
display(f"book: {booksDf.columns}\n")

"customuser: Index(['id', 'password', 'last_login', 'username', 'first_name', 'last_name',\n       'is_active', 'is_staff', 'is_superuser', 'date_joined'],\n      dtype='object')\n"

"rating: Index(['id', 'ISBN', 'rating', 'user_id'], dtype='object')\n"

"book: Index(['id', 'title', 'author', 'publisher', 'book_slug', 'ISBN',\n       'year_of_publication', 'total_rating', 'img_s', 'img_m', 'img_l'],\n      dtype='object')\n"

In [ ]:
ratings_with_name = ratingsDf.merge(booksDf,on='ISBN')

In [ ]:
ratingsDf['user_id'].value_counts().reset_index().columns

Index(['index', 'user_id'], dtype='object')

In [ ]:
ratings_with_name.head()

,id_x,ISBN,rating,user_id,id_y,title,author,publisher,book_slug,year_of_publication,total_rating,img_s,img_m,img_l
0,1149781,034545104X,0,276725,2967,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,FleshTonesANovel,2002,6.28571428571429,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,1165214,034545104X,5,2313,2967,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,FleshTonesANovel,2002,6.28571428571429,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,1177314,034545104X,0,6543,2967,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,FleshTonesANovel,2002,6.28571428571429,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,1185869,034545104X,5,8680,2967,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,FleshTonesANovel,2002,6.28571428571429,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,1190575,034545104X,9,10314,2967,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,FleshTonesANovel,2002,6.28571428571429,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [ ]:
ratings_with_name.drop(columns=["ISBN","img_s","img_m"],axis=1,inplace=True)
customuserDf.rename(columns={"id":"user_id"}, inplace=True)

In [ ]:
ratings_with_name["user_id"] = ratings_with_name["user_id"].astype('int')

In [ ]:
complete_df = ratings_with_name.merge(customuserDf, on="user_id")
complete_df.head()

,id_x,rating,user_id,id_y,title,author,publisher,book_slug,year_of_publication,total_rating,img_l,password,last_login,username,first_name,last_name,is_active,is_staff,is_superuser,date_joined
0,1149781,0,276725,2967,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,FleshTonesANovel,2002,6.28571428571429,http://images.amazon.com/images/P/034545104X.0...,pbkdf2_sha256$390000$0cPpAdrcdSrakYRxFdbzvn$Mz...,None,QTIuzEw0,Hai,Nguyen,1,0,0,2023-04-05 01:52:47.894190
1,1165214,5,2313,2967,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,FleshTonesANovel,2002,6.28571428571429,http://images.amazon.com/images/P/034545104X.0...,pbkdf2_sha256$390000$0cPpAdrcdSrakYRxFdbzvn$Mz...,None,8TDT7eA7,Hai,Nguyen,1,0,0,2023-04-05 01:52:47.894190
2,1165237,9,2313,2997,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,Tor Books,EndersGameEnderWigginsSagaPaperback,1986,Null,http://images.amazon.com/images/P/0812533550.0...,pbkdf2_sha256$390000$0cPpAdrcdSrakYRxFdbzvn$Mz...,None,8TDT7eA7,Hai,Nguyen,1,0,0,2023-04-05 01:52:47.894190
3,1165232,8,2313,2993,In Cold Blood (Vintage International),TRUMAN CAPOTE,Vintage,InColdBloodVintageInternational,1994,Null,http://images.amazon.com/images/P/0679745580.0...,pbkdf2_sha256$390000$0cPpAdrcdSrakYRxFdbzvn$Mz...,None,8TDT7eA7,Hai,Nguyen,1,0,0,2023-04-05 01:52:47.894190
4,1165207,9,2313,404,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,HarperCollins,DivineSecretsoftheYaYaSisterhoodANovel,1996,7.53191489361702,http://images.amazon.com/images/P/0060173289.0...,pbkdf2_sha256$390000$0cPpAdrcdSrakYRxFdbzvn$Mz...,None,8TDT7eA7,Hai,Nguyen,1,0,0,2023-04-05 01:52:47.894190


In [ ]:
x = complete_df.groupby('user_id').count()['rating']>200
knowledgable_users = x[x].index

In [ ]:
filtered_rating = complete_df[complete_df['user_id'].isin(knowledgable_users)]

In [ ]:
y = filtered_rating.groupby('title').count()['rating']>=50
famous_books = y[y].index

In [ ]:
final_ratings =  filtered_rating[filtered_rating['title'].isin(famous_books)]

In [ ]:
testdf = final_ratings.sort_values("rating",ascending=False)
testdf[["title",'rating']].groupby("title").mean().sort_values("rating",ascending=False)

,rating
title,
The Little Prince,5.120000
Harry Potter and the Sorcerer's Stone (Book 1),4.893333
Harry Potter and the Goblet of Fire (Book 4),4.471545
Harry Potter and the Prisoner of Azkaban (Book 3),4.414815
Harry Potter and the Order of the Phoenix (Book 5),4.262136
...,...
Pleading Guilty,0.548387
Deck the Halls (Holiday Classics),0.508772
Wild Animus,0.442424


In [ ]:
pt = final_ratings.pivot_table(index='title',columns='user_id'
                          ,values='rating')
pt

user_id,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Bend in the Road,0.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,9.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [ ]:
pt.fillna(0,inplace=True)
pt

user_id,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity 

In [ ]:
similarity_score = cosine_similarity(pt)
similarity_score.shape

(706, 706)

In [ ]:
def recommend(book_name):
    index = np.where(pt.index==book_name)[0][0]
    similar_books = sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1], reverse=True)[0:6]
    
    data = []
    
    for i in similar_books:
        item = []
        temp_df = booksDf[booksDf['title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('title')['title'].values))
        item.extend(list(temp_df.drop_duplicates('title')['author'].values))
        item.extend(list(temp_df.drop_duplicates('title')['img_l'].values))
        
        data.append(item)
    return data

In [24]:
recommend("Year of Wonders")

[['Year of Wonders',
  'Geraldine Brooks',
  'http://images.amazon.com/images/P/0142001430.01.LZZZZZZZ.jpg'],
 ['The Eight',
  'Katherine Neville',
  'http://images.amazon.com/images/P/0345366239.01.LZZZZZZZ.jpg'],
 ['A Heartbreaking Work of Staggering Genius',
  'Dave Eggers',
  'http://images.amazon.com/images/P/0375725784.01.LZZZZZZZ.jpg'],
 ['Bridget Jones: The Edge of Reason',
  'Helen Fielding',
  'http://images.amazon.com/images/P/0140298479.01.LZZZZZZZ.jpg'],
 ["Drowning Ruth (Oprah's Book Club)",
  'CHRISTINA SCHWARZ',
  'http://images.amazon.com/images/P/0345439104.01.LZZZZZZZ.jpg'],
 ['Midwives: A Novel',
  'Chris Bohjalian',
  'http://images.amazon.com/images/P/0375706771.01.LZZZZZZZ.jpg']]